## Previsão de emissão de poluente para classificação de veículos

##### O time de negócio da marca Piat deseja agrupar os veículos da empresa que tenham uma menor pegada de carbono, facilitando assim ações de marketing com este subgrupo.


*   **User Story:** "Como diretor de marketing, quero realizar a previsão de emissão de CO2 para saber o acumulado de emissão de poluente de cada veículo e decidir se ele pertence à classe de veículos sustentáveis da Piat".

###### O Co2 predictor é um software desenvolvido para gerar estes resultados para o time, a partir de um modelo de Machine Learning de Regressão Linear. 

In [ ]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 2.1 MB/s 
     |████████████████████████████████| 255.9 MB 42 kB/s 
     |████████████████████████████████| 139 kB 42.9 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11953 sha256=d91e61f2c3b8097ccb450489fb4a09dced5037cc05c8d39a4a14052ab0645fbd
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
# Pacotes
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pickle

In [ ]:
class Loader:

    def load_data(self, url: str):
        """ Carrega o arquivo e retorna um DataFrame.
        :url: string com  o nome/endereço do file
        """  
        return pd.read_csv(url)

In [ ]:
class PreProcessor:

    def pre_process_data(self, data, percentual_teste, redundant_cols: list, seed=42):
        """ Realiza o pré processamento dos dados.
        :data: dataframe
        :percentual_teste: percentual dos dados definido para teste
        :seed: semente randômica
        :redundant_cols: lista de colunas que podem ser removidas do dataframe
        """

        # feature selection
        data.drop(redundant_cols, axis=1, inplace=True)

        # divisão em treino e teste
        X_train, X_test, Y_train, Y_test = self.__preparar_holdout(data,
                                                                  percentual_teste,
                                                                  seed)

        return (X_train, X_test, Y_train, Y_test)

    def __preparar_holdout(self, data, percentual_teste, seed):
        """ Divide os dados em treino e teste usando o método holdout.
        :data: dataframe
        :percentual_teste: percentual dos dados definido para teste
        :seed: semente randômica
        """
        data = data.values
        data = data.astype('float32')
        X, Y = data[:, :-1], data[:, -1]
        return train_test_split(X, Y, test_size=percentual_teste, random_state=seed)


In [ ]:
class MLModel:

    def select_best_model(self, cv, X_train, Y_train):
        """ Utiliza AutoML para identificar o melhor modelo de regressão.
        :cv: define a validação cruzada
        :X_train: features da base de treino
        :Y_treino: variável target da base de treino
        """

        # define busca do melhor modelo de regressão
        model = TPOTRegressor(generations=5, population_size=50, scoring='r2', cv=cv, verbosity=2, random_state=1, n_jobs=-1)
        model.fit(X_train, Y_train)
        model.export('best_model.py')
        
        # display resultados do AutoML
        resultado = pd.DataFrame(model.evaluated_individuals_)
        resultado.columns = list(map(lambda x: x[0], resultado.columns.str.split('(')))
        return print(resultado.T)

    def model_trainning(self, X_train, Y_train):
        """ Cria pipeline de treinamento do melhor modelo encontrado na etapa de search.
        :X_train: features da base de treino
        :Y_treino: variável target da base de treino
        """
        best_pipeline = ExtraTreesRegressor(bootstrap=False, max_features=0.25, min_samples_leaf=1, min_samples_split=5, n_estimators=100)
        best_pipeline = best_pipeline.fit(X_train, Y_train)
        return best_pipeline 

In [ ]:
class MLEvaluator:

    def avaliar_r2_score(self, best_pipeline, X_test, Y_test):
        """ Avalia o modelo fazendo uma predição com base na métria r-quadrado
        :X_teste: features da base de teste
        :Y_teste: variável target da base de teste
        :modelo: modelo treinado
        """
        results = best_pipeline.score(X_test, Y_test)
        return print(results)

In [ ]:
class ModelExport:

    def export_best_model(self, best_pipeline):
        """ Exporta o modelo treinado em um arquivo de formato pickle
        :modelo: modelo treinado
        """
        artifact_pkl_filename = 'model.pkl'

        local_path = artifact_pkl_filename
        with open(local_path, 'wb') as model_file:
          return pickle.dump(best_pipeline, model_file)

In [ ]:
# Instanciação das Classes
loader = Loader()
pre_processor = PreProcessor()
model = MLModel()
performance_evaluator = MLEvaluator()
export_model = ModelExport()

In [ ]:
# Parâmetros
url_dados = ('https://raw.githubusercontent.com/gerolaleticia/Pyrentena-a-dataset-per-week/master/Semana%203%20%20-%20Dataset%20CO2/FuelConsumptionCo2.csv')
redundant_cols = ['MODELYEAR','MAKE','MODEL','VEHICLECLASS','TRANSMISSION','FUELTYPE']
percentual_teste = 0.2
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1) 

In [ ]:
# Execução do pipeline de treinamento

# carga
data = loader.load_data(url_dados) 

X_train, X_test, Y_train, Y_test = pre_processor.pre_process_data(data, percentual_teste, redundant_cols) 

# Busca e seleção do melhor modelo com Auto ML
best_model_report = model.select_best_model(cv, X_train, Y_train)

# Treinamento do melhor modelo
best_pipeline = model.model_trainning(X_train, Y_train)

# Resultados de performance considerando r2 score
performance_evaluator.avaliar_r2_score(best_pipeline, X_test, Y_test)

# Export do modelo treinado
loaded_pkl_model = export_model.export_best_model(best_pipeline)

### Deploy

Pipeline de implementação do modelo treinado: recebe novos dados e salva as predições em um novo file csv para consumo da equipe de negócio.

In [ ]:
class PreProcessorDeploy:

    def pre_process_new_data(self, data, redundant_cols: list):
        """ Realiza o pré processamento dos dados.
        :data: dataframe com novos dados
        :redundant_cols: lista de colunas que podem ser removidas do dataframe
        """

        # feature selection
        data.drop(redundant_cols, axis=1, inplace=True)
        del data['Unnamed: 0']

        data = data.values
        data = data.astype('float32')
        return data

In [ ]:
class LoadModel:

    def load_trained_model(self, artifact_pkl_filename: str):
        """ Baixa o modelo salvado em arquivo pickle para aplicação nos novos dados. 
        :artifact_pkl_filename: nome do arquivo pkl
        """
        loaded_pkl_model = pickle.load(open(artifact_pkl_filename, 'rb'))
        return loaded_pkl_model

In [ ]:
class Output:

    def create_output_dataframe(self, predicoes):
        """ Gera o dataframe de predições e concatena com os novos dados, salvando o output final em um arquivo csv.
        :predcicoes: dados de predição que o modelo gerou
        """

        output = pd.DataFrame(predicoes, columns = ['CO2_predictions'])
        output = pd.concat([new_data, output], axis=1)
        output.to_csv('predicoes.csv')
        return output.head()

In [ ]:
# Parâmetros
new_file = 'brand_new_data.csv'
pkl_model_file = 'model.pkl'

In [ ]:
# Instanciação das Classes
preprocess_deploy = PreProcessorDeploy()
generate_output = Output()
load_model = LoadModel()

In [ ]:
# Execução do pipeline de produção

# carga
new_data = loader.load_data(new_file) 

# processamento dos novos dados
X = preprocess_deploy.pre_process_new_data(new_data, redundant_cols)

# load do modelo treinado
loaded_pkl_model = load_model.load_trained_model(pkl_model_file)

# aplicação do modelo já treinado
predicoes = loaded_pkl_model.predict(X)

# geração e export do arquivo de predições em csv
generate_output.create_output_dataframe(predicoes)

,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2_predictions
0,2.0,4,9.9,6.7,8.5,33,196.375833
1,2.4,4,11.2,7.7,9.6,29,221.470833
2,1.5,4,6.0,5.8,5.9,48,138.004167
3,3.5,6,12.7,9.1,11.1,25,256.171667
4,3.5,6,12.1,8.7,10.6,27,245.403333
